In [ ]:
# %% [markdown]
# # Entraînement du modèle de génération
# 
# Ce notebook entraîne un modèle de langage sur des données d'art algorithmique.

# %%
import sys
sys.path.append('..')

from src.model_training import ModelTrainer
from config.settings import config
import torch
from pathlib import Path



In [ ]:
# %%
# Vérifier la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")



In [ ]:
# %%
# Initialiser l'entraîneur
trainer = ModelTrainer(config)



In [ ]:
# %%
# Préparer les données
data_dir = Path(config.PROCESSED_DIR)
if not data_dir.exists():
    print(f"Le répertoire {data_dir} n'existe pas.")
    print("Exécutez d'abord le notebook 01_data_exploration.ipynb ou le script prepare-data.")
else:
    # Charger les données SVG
    print("Préparation des données SVG...")
    svg_dataset = trainer.prepare_data(data_dir, "svg")
    
    print(f"Données d'entraînement SVG: {len(svg_dataset['train'])} exemples")
    print(f"Données de validation SVG: {len(svg_dataset['test'])} exemples")
    
    

In [ ]:
# %%
    # Entraîner le modèle SVG
    print("\nEntraînement du modèle SVG...")
    svg_trainer = trainer.train(
        svg_dataset["train"],
        svg_dataset["test"],
        "svg"
    )
    
    

In [ ]:
# %%
    # Évaluer le modèle
    eval_results = svg_trainer.evaluate()
    print(f"Perte d'évaluation: {eval_results['eval_loss']:.4f}")
    
    

In [ ]:
# %%
    # Préparer les données G-code
    print("\nPréparation des données G-code...")
    gcode_dataset = trainer.prepare_data(data_dir, "gcode")
    
    print(f"Données d'entraînement G-code: {len(gcode_dataset['train'])} exemples")
    print(f"Données de validation G-code: {len(gcode_dataset['test'])} exemples")
    
    

In [ ]:
# %%
    # Entraîner le modèle G-code
    print("\nEntraînement du modèle G-code...")
    trainer_gcode = ModelTrainer(config)  # Nouvelle instance
    gcode_trainer = trainer_gcode.train(
        gcode_dataset["train"],
        gcode_dataset["test"],
        "gcode"
    )
    
    

In [ ]:
# %%
    # Évaluer le modèle G-code
    gcode_eval = gcode_trainer.evaluate()
    print(f"Perte d'évaluation G-code: {gcode_eval['eval_loss']:.4f}")



In [ ]:
# %%
# Option: Entraîner un modèle multi-format
print("\nOption: Entraînement d'un modèle multi-format...")
trainer_multi = ModelTrainer(config)
multi_trainer = trainer_multi.train_multi_format(data_dir, data_dir)



In [ ]:
# %%
# Sauvegarder les modèles
print("\nModèles sauvegardés dans:")
print("  - models/svg_model_final/")
print("  - models/gcode_model_final/")
print("  - models/multi_format_model_final/")